In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.impute import KNNImputer 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from fancyimpute import KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import  f1_score,precision_score,recall_score,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from gensim.models import Word2Vec, KeyedVectors
import nltk
from gensim.test.utils import common_texts
import regex
import re
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE
from keras import backend as K

smote = SMOTE(sampling_strategy='not majority')

import warnings
warnings.filterwarnings('ignore')

spam_df = pd.read_csv('C:/spam_or_not_spam.csv')

spam_df

spam_df.isnull().sum()

spam_df.dropna()

def make_sequences(inp_text, tokenizer, train=True, max_seq_length=None):
    sequences = tokenizer.texts_to_sequences(inp_text)
    
    if train == True:
        max_seq_length = np.max(list(map(lambda z: len(z), sequences)))
    
    sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    
    return sequences

def tokenize_df(df):
    df = df.copy()
     
    y = df['label']
    X = (df['email']).astype("str")
   
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=True, random_state=1)
    
    # Makes the tokenizer
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=30000)
    
    # Tokenizer fitting
    tokenizer.fit_on_texts(X_train)
    tokenizer.fit_on_texts(X_test)
    
    #texts to sequences
    X_train = make_sequences(X_train, tokenizer, train=True)
    X_test = make_sequences(X_test, tokenizer, train=False, max_seq_length=X_train.shape[1])
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = tokenize_df(spam_df)

X_train

X_test

y_test

y_train

X_train.shape

#define metric functions to use in keras

def recall_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_metric(y_true, y_pred):
    precision = precision_metric(y_true, y_pred)
    recall = recall_metric(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

inputs = tf.keras.Input(shape=(4935,))

embedding = tf.keras.layers.Embedding(
    input_dim=20000,  #30000 was here
    output_dim=200 #64 was here ,write the changes in these parameters of the neural network and the results
)(inputs)

flatten = tf.keras.layers.Flatten()(embedding)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        #'accuracy',
         f1_metric,
         precision_metric, 
         recall_metric
        #tf.keras.metrics.AUC(name='auc')
         
    ]
)


print(model.summary())

history = model.fit(
    X_train,
    y_train,
    validation_split=0.25,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='f1_metric',#consider to change the monitor variable for f1 or recall or precision
            patience=3,#after so non changes to the monitor value we stop
            restore_best_weights=True
        )
    ]
)

metric_results = model.evaluate(
    x=X_train, y=y_train,verbose=0
)

metric_results



NameError: name 'pd' is not defined